In [1]:
import gym
import numpy as np
from tqdm import tqdm
import torch
import wandb
from copy import deepcopy

from fractal_zero.config import FMCConfig
from fractal_zero.search.fmc import FMC
from fractal_zero.models.prediction import FullyConnectedPredictionModel
from fractal_zero.models.policies.cartpole_policy import CartpolePolicy
from fractal_zero.vectorized_environment import (
    RayVectorizedEnvironment,
    VectorizedDynamicsModelEnvironment,

)
from fractal_zero.trainers.online import OnlineFMCPolicyTrainer


/Users/dyllan/miniconda3/envs/fz/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
NUM_WALKERS = 64

class CartpolePolicy(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.net = torch.nn.Sequential(
            torch.nn.Linear(4, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 2),
            # torch.nn.Sigmoid(),  # keep MSE from exploding
        )

    def forward(self, observations, with_randomness: bool = False):
        observations = torch.tensor(observations).float()

        y = self.net(observations)

        if with_randomness:
            # center = embeddings.std()
            # center = y.var()
            # centered_uniform_noise = (torch.rand_like(y) * center) - (center / 2)
            # y += centered_uniform_noise
            raise NotImplementedError

        return y

    def parse_action(self, actions):
        # actions = torch.where(actions > 0.5, 1, 0).flatten()
        actions = actions.argmax(-1)
        l = actions.tolist()
        return l

policy_model = CartpolePolicy()

In [3]:
# optimizer = torch.optim.SGD(policy_model.parameters(), lr=0.01, weight_decay=1e-4)
optimizer = torch.optim.Adam(policy_model.parameters(), lr=0.03, weight_decay=1e-4)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)

loss_func = torch.nn.functional.cross_entropy
policy_trainer = OnlineFMCPolicyTrainer("CartPole-v0", policy_model, optimizer, NUM_WALKERS, loss_spec=loss_func, use_ray=False)

/Users/dyllan/miniconda3/envs/fz/lib/python3.10/site-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/Users/dyllan/miniconda3/envs/fz/lib/python3.10/site-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/Users/dyllan/miniconda3/envs/fz/lib/python3.10/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [4]:
wandb.init(project="fz-policy-trainer-game-tree")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dyllan. Use `wandb login --relogin` to force relogin
/Users/dyllan/miniconda3/envs/fz/lib/python3.10/site-packages/wandb/sdk/lib/ipython.py:46: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display  # type: ignore


In [5]:
num_episodes = 100
train_steps_per_episode = 10
eval_every = 20
max_steps = 200

best_total_rewards = float("-inf")
best_model = None

for _ in range(num_episodes):
    policy_trainer.generate_episode_data(max_steps)
    print("best path reward", policy_trainer.fmc.tree.best_path.total_reward)

    for i in range(train_steps_per_episode):
        policy_trainer.train_on_latest_episode()

        if i % eval_every == 0:
            total_rewards = policy_trainer.evaluate_policy(max_steps)

            if total_rewards > best_total_rewards:
                best_total_rewards = total_rewards
                best_model = deepcopy(policy_model)
                
                # torch.save(best_model, "models/best_cartpole_policy.pth")

    # lr_scheduler.step()

/Users/dyllan/Code/fractal-zero/fractal_zero/vectorized_environment.py:176: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:204.)
  states = self.observation_encoder(observations)
/Users/dyllan/miniconda3/envs/fz/lib/python3.10/site-packages/gym/envs/classic_control/cartpole.py:179: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  logger.warn(


best path reward 200.0
best path reward 200.0
best path reward 200.0
best path reward 199.0
best path reward 200.0
best path reward 200.0
best path reward 200.0
best path reward 199.0
best path reward 200.0
best path reward 200.0
best path reward 184.0
best path reward 200.0
best path reward 200.0
best path reward 200.0
best path reward 200.0
best path reward 195.0
best path reward 180.0
best path reward 200.0
best path reward 200.0
best path reward 200.0
best path reward 200.0
best path reward 200.0
best path reward 200.0
best path reward 184.0
best path reward 163.0
best path reward 200.0


In [ ]:
policy_trainer.fmc.clone_receives